<a href="https://colab.research.google.com/github/spirosChv/smartNetsWorkshop/blob/main/brian2/Playing_with_dendrites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
#@title Install dependencies (might take a few seconds)
!pip install brian2 --quiet

In [ ]:
#@title Imports and settings
import brian2 as b
from brian2.units import *

b.prefs.codegen.target = 'numpy' # Improves performance significantly here
b.start_scope()    # allows running separate simulations in the same notebook

# @title Figure settings
blue = '#005c94ff'
green = '#338000ff'
orange = '#ff6600ff'
params = {
          "legend.fontsize": 10,
          "legend.handlelength": 1.5,
          "legend.edgecolor": 'inherit',
          "legend.columnspacing": 0.8,
          "legend.handletextpad": 0.5,
          "axes.labelsize": 10,
          "axes.titlesize": 11, 
          "axes.spines.right": False,
          "axes.spines.top": False,
          "xtick.labelsize": 10,
          "ytick.labelsize": 10,
          'mathtext.default': 'regular',
          'lines.markersize': 3,
          'lines.linewidth': 1.25,
          'grid.color': "#d3d3d3",
          'text.antialiased': True,
          'lines.antialiased': True,
          'figure.dpi': 150,
          'axes.prop_cycle': b.cycler(color=[blue, orange, green])
          }

b.rcParams.update(params)

## Build model
> Let's first creat a 3-compartment toy model with passive dendrites:

In [ ]:
eqs = """
    dV_soma/dt = (gL_soma * (EL_soma-V_soma) + I_soma) / C_soma  :volt
    I_soma = I_ext_soma + I_basal_soma  + I_apical_soma   :amp
    I_ext_soma  :amp
    I_apical_soma = (V_apical-V_soma) * g_apical_soma  :amp
    I_basal_soma = (V_basal-V_soma) * g_basal_soma  :amp

    dV_apical/dt = (gL_apical * (EL_apical-V_apical) + I_apical) / C_apical  :volt
    I_apical = I_ext_apical + I_soma_apical  + I_NMDA_cortex_apical + I_AMPA_cortex_apical  :amp
    I_ext_apical  :amp
    I_AMPA_cortex_apical = g_AMPA_cortex_apical * (E_AMPA-V_apical) * s_AMPA_cortex_apical  :amp
    ds_AMPA_cortex_apical/dt = -s_AMPA_cortex_apical / t_AMPA_decay_cortex_apical  :1
    I_NMDA_cortex_apical = g_NMDA_cortex_apical * (E_NMDA-V_apical) * s_NMDA_cortex_apical / (1 + Mg * exp(-alpha*(V_apical/mV+gamma)) / beta)  :amp
    ds_NMDA_cortex_apical/dt = -s_NMDA_cortex_apical/t_NMDA_decay_cortex_apical  :1
    I_soma_apical = (V_soma-V_apical) * g_soma_apical  :amp

    dV_basal/dt = (gL_basal * (EL_basal-V_basal) + I_basal) / C_basal  :volt
    I_basal = I_ext_basal + I_soma_basal   :amp
    I_ext_basal  :amp
    I_soma_basal = (V_soma-V_basal) * g_soma_basal  :amp"""

params = {
    'C_apical': 70.68583471 * pfarad,
    'C_basal': 42.41150082 * pfarad,
    'C_soma': 58.90486225 * pfarad,
    'EL_apical': -70. * mvolt,
    'EL_basal': -70. * mvolt,
    'EL_soma': -70. * mvolt,
    'E_AMPA': 0. * volt,
    'E_NMDA': 0. * volt,
    'Mg': 1.0,
    'alpha': 0.062,
    'beta': 3.57,
    'gL_apical': 3.53429174 * nsiemens,
    'gL_basal': 2.12057504 * nsiemens,
    'gL_soma': 2.94524311 * nsiemens,
    'g_AMPA_cortex_apical': 1. * nsiemens,
    'g_NMDA_cortex_apical': 1. * nsiemens,
    'g_apical_soma': 10. * nsiemens,
    'g_basal_soma': 10. * nsiemens,
    'g_soma_apical': 10. * nsiemens,
    'g_soma_basal': 10. * nsiemens,
    'gamma': 0,
    't_AMPA_decay_cortex_apical': 2. * msecond,
    't_NMDA_decay_cortex_apical': 60. * msecond
    }

# create a Brian NeuronGroup and inialize v rest
pyr_group = b.NeuronGroup(3, model=eqs, method='euler',
                          threshold='V_soma > -40*mV',
                          reset='V_soma = -50*mV',
                          refractory=3*ms,
                          namespace=params)
pyr_group.V_soma = -70. * mvolt
pyr_group.V_apical = -70. * mvolt
pyr_group.V_basal = -70. * mvolt


> *Our model now looks somewhat like this:*
>
><p align="center">
    <img src="https://github.com/mpgl/dendrify-paper/blob/main/graphics/1.png?raw=true" alt="model" width="25%">
></p>

## Panel b | Somatodendritic attenuation
> *We are going to apply depolarizing current injections (400 ms pulses of 100 pA at -70 mV baseline voltage) individually to each compartment and record the voltage responses of all compartments.*

In [ ]:
# Set monitors to record membrane voltages
M = b.StateMonitor(pyr_group, ["V_soma", "V_apical", "V_basal"], record=True)

# Set input current amplitude
I = 100 * pA

# First 100 ms -> no input
b.run(100*ms)

# Next 400 ms -> square pulse of I amplitude
pyr_group.I_ext_soma[0] = I
pyr_group.I_ext_apical[1] = I
pyr_group.I_ext_basal[2] = I
b.run(400*ms)

# Next 200 ms -> no input to let membranes return to Vrest
pyr_group.I_ext_soma[0] = 0 * pA
pyr_group.I_ext_apical[1] = 0 * pA
pyr_group.I_ext_basal[2] = 0 * pA
b.run(200*ms)

In [ ]:
# @title Plot voltage traces
time = M.t/ms
vs = M.V_soma/mV
va = M.V_apical/mV
vb = M.V_basal/mV

fig, axes = b.subplots(1, 3, figsize=[6, 2.5], sharex=True, sharey=True)
ax0, ax1, ax2 = axes

for id, ax in enumerate([ax0, ax1, ax2]):
    for v in vs, va, vb:
        ax.plot(time, v[id])

ax0.set_ylabel('Voltage (mV)')
ax1.set_xlabel('Time (ms)')

ax0.set_title("Input \u2192 soma")
ax1.set_title("Input \u2192 apical")
ax2.set_title("Input \u2192 basal")

fig.tight_layout()

> *Notice the electrical segmentation caused by the attenuation of currents traveling along the somatodendritic axis. Although this property may seem undesirable, it allows each compartment to operate semi-autonomously from the others, thus multiple input integration sites can coexist within a single neuron.*

## Panels c-d | Dendritic integration
> *Here, we will activate simultaneously an increasing number of synapses on the apical dendrite and measure how dendritic integration affects the somatic output. For better performance, we will perform this experiment in a vectorized manner; we will create multiple instances of the same neuron, each of which will be connected to a different number of synapses.*

In [ ]:
# create a new Brian NeuronGroup and link it to the existing NeuronModel
b.start_scope()    # allows running separate simulations in the same notebook
Nsyn = 35    # max number of synapses
pyr_group2 = b.NeuronGroup(Nsyn, model=eqs, method='euler',
                           threshold='V_soma > -40*mV',
                           reset='V_soma = -50*mV',
                           refractory=3*ms,
                           namespace=params)
pyr_group2.V_soma = -70. * mvolt
pyr_group2.V_apical = -70. * mvolt
pyr_group2.V_basal = -70. * mvolt


# create a source of presynaptic input (N "neurons" being activated at 50 ms)
spiketimes = [(50*ms) for n in range(Nsyn)]
I = b.SpikeGeneratorGroup(Nsyn, range(Nsyn), spiketimes)


# connect an increasing number of synapses to each neuron instance
synaptic_effect = "s_AMPA_cortex_apical += 1.0; s_NMDA_cortex_apical += 1.0"
S = b.Synapses(I, pyr_group2, on_pre=synaptic_effect)
S.connect('j >= i')    # j, i -> postsynaptic, presynaptic indices respectively


# Set monitors to record membrane voltages and run simulation
M = b.StateMonitor(pyr_group2, ["V_soma", "V_apical", "V_basal"], record=True)
b.run(400*ms)

vs_control = M.V_soma/mV
va_control = M.V_apical/mV
vb_control = M.V_basal/mV

> *Also, to test the effect of the dendritic NMDA-dependent nonlinearities we will repeat the same exact protocol without activating the NMDA synaptic component (see line 16).*

In [ ]:
# create a new Brian NeuronGroup and link it to the existing NeuronModel
b.start_scope()    # allows running separate simulations in the same notebook
Nsyn = 35    # max number of synapses
pyr_group3 = b.NeuronGroup(Nsyn, model=eqs, method='euler',
                           threshold='V_soma > -40*mV',
                           reset='V_soma = -50*mV',
                           refractory=3*ms,
                           namespace=params)
pyr_group3.V_soma = -70. * mvolt
pyr_group3.V_apical = -70. * mvolt
pyr_group3.V_basal = -70. * mvolt


# create a source of presynaptic input (N "neurons" being activated at 50 ms)
spiketimes = [(50*ms) for n in range(Nsyn)]
I = b.SpikeGeneratorGroup(Nsyn, range(Nsyn), spiketimes)


# connect an increasing number of synapses to each neuron instance
synaptic_effect = "s_AMPA_cortex_apical += 1.0"
S = b.Synapses(I, pyr_group3, on_pre=synaptic_effect)
S.connect('j >= i')    # j, i -> postsynaptic, presynaptic indices respectively


# Set monitors to record membrane voltages and run simulation
M = b.StateMonitor(pyr_group3, ["V_soma", "V_apical", "V_basal"], record=True)
b.run(400*ms)

time = M.t/ms
vs_ampa = M.V_soma/mV
va_ampa = M.V_apical/mV
vb_ampa = M.V_basal/mV

In [ ]:
# @title Plot somatic responses
fig, axes = b.subplots(1, 2, figsize=[5, 2.5], sharex=True, sharey=True)
ax0, ax1 = axes
secret_ax = fig.add_subplot(111, frameon=False)
secret_ax.tick_params(labelcolor='none', which='both', bottom=False, left=False)

for i in range(Nsyn):
    if (i+1)%5 == 0:
        ax0.plot(time, vs_control[i], c='crimson')
        ax1.plot(time, vs_ampa[i], c='#888a85ff')
ax0.set_ylabel('Voltage (mV)')
ax0.set_title("AMPA & NMDA")
ax1.set_title("AMPA only")
secret_ax.set_xlabel('Time (ms)')
fig.tight_layout()

> *Notice the huge effect of the dendritic NMDA currents on both the amplitude and the kinetics of the somatic voltage responses. For visual clarity, the data shown here represent a range of 5-35 synapses with step 5.*

In [ ]:
# @title Plot input-output function
# In this example, due to noise we cannot accurately estimate the uEPSP amplitude.
# So we infer it by deviding a compound EPSP from 5 synapses by 5. We can do this
# because up until 5 synapses the IO relationship is very close to linear.
peaks_control = [max(v)+70 for v in vs_control]
unitary_control = peaks_control[4]/5
expected_control = b.linspace(1, 35, 35) * unitary_control

peaks_ampa = [max(v)+70 for v in vs_ampa]
unitary_ampa = peaks_ampa[4]/5
expected_ampa = b.linspace(1, 35, 35) * unitary_ampa

fig, ax = b.subplots(figsize=[3, 3])
ax.plot(expected_control, peaks_control, c='crimson', label='AMPA & NMDA')
ax.plot(expected_ampa, peaks_ampa, c='#888a85ff', label='AMPA only' )
ax.plot(range(20), range(20), c='black', ls=':', label='linear')
ax.legend()
ax.set_xlabel('Expected EPSP (mV)')
ax.set_ylabel('Measured EPSP (mV)')
fig.tight_layout()

> *Notice the shift from supralinear to sublinear integration when NMDARs are blocked. This happens because synaptic currents are susceptible to the decrease in driving force as dendritic voltage approaches the AMPA reversal potential (EAMPA = 0 mV).*